# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [1]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [1]:
import tensorflow as tf
from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
)
import os
from datetime import datetime

## Check configuration

In [2]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.1.0-rc2-17-ge5bf8de410 2.1.0


In [3]:
print(tf.keras.__version__)

2.2.4-tf


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [5]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Train the model on AI Platform Training (for production)

In [20]:
# train on GCP
model_name='tf_bert_classification'
os.environ['JOB_NAME'] = model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['RUNTIME_VERSION'] = '2.1'
os.environ['PYTHON_VERSION'] = '3.7'
os.environ['TRAINER_PACKAGE_PATH'] = os.environ['PYTHONPATH']+'/model'
os.environ['MAIN_TRAINER_MODULE'] = 'model.'+model_name+'.task'

os.environ['EPOCHS'] = '1' 
os.environ['STEPS_PER_EPOCH_TRAIN'] = '1' 
os.environ['BATCH_SIZE_TRAIN'] = '32' 
os.environ['STEPS_PER_EPOCH_EVAL'] = '1' 
os.environ['BATCH_SIZE_EVAL'] = '64'
os.environ['PACKAGE_STAGING_PATH'] = 'gs://'+os.environ['BUCKET_STAGING_NAME']
os.environ['INPUT_EVAL_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['INPUT_TRAIN_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['OUTPUT_DIR'] = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['PRETRAINED_MODEL_DIR']= 'gs://'+os.environ['BUCKET_NAME']+'/pretrained_model/bert-base-multilingual-uncased'

In [21]:
%%bash
# Use Cloud Machine Learning Engine to train the model on GCP
gcloud ai-platform jobs submit training $JOB_NAME \
        --scale-tier basic \
        --python-version $PYTHON_VERSION \
        --runtime-version $RUNTIME_VERSION \
        --module-name=$MAIN_TRAINER_MODULE \
        --package-path=$TRAINER_PACKAGE_PATH \
        --staging-bucket=$PACKAGE_STAGING_PATH \
        --region=$REGION \
        -- \
        --epochs=$EPOCHS \
        --steps_per_epoch_train=$STEPS_PER_EPOCH_TRAIN \
        --batch_size_train=$BATCH_SIZE_TRAIN \
        --steps_per_epoch_eval=$STEPS_PER_EPOCH_EVAL \
        --batch_size_eval=$BATCH_SIZE_EVAL \
        --input_eval_tfrecords=$INPUT_EVAL_TFRECORDS \
        --input_train_tfrecords=$INPUT_TRAIN_TFRECORDS \
        --output_dir=$OUTPUT_DIR \
        --pretrained_model_dir=$PRETRAINED_MODEL_DIR \
        --verbosity_level='DEBUG' \
#        --stream-logs

jobId: tf_bert_classification_2020_05_04_160650
state: QUEUED


Job [tf_bert_classification_2020_05_04_160650] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe tf_bert_classification_2020_05_04_160650

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs tf_bert_classification_2020_05_04_160650


## Train the model on AI Platform Training using a config file (for production)

In [82]:
model_name='tf_bert_classification'
os.environ['JOB_NAME'] = model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['TRAINER_PACKAGE_PATH'] = os.environ['PYTHONPATH']+'/model'
os.environ['MAIN_TRAINER_MODULE'] = 'model.'+model_name+'.task'
os.environ['CONFIG']= os.environ['DIR_PROJ'].split('src')[0]+'deployment/training/tf_bert_classification/standard.yaml' #or custom.yaml or standard.yaml
os.environ['EPOCHS'] = '2' 
os.environ['STEPS_PER_EPOCH_TRAIN'] = '5' 
os.environ['BATCH_SIZE_TRAIN'] = '32' 
os.environ['STEPS_PER_EPOCH_EVAL'] = '1' 
os.environ['BATCH_SIZE_EVAL'] = '64'Salut
os.environ['PACKAGE_STAGING_PATH'] = 'gs://'+os.environ['BUCKET_STAGING_NAME']
os.environ['INPUT_EVAL_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['INPUT_TRAIN_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['OUTPUT_DIR'] = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['PRETRAINED_MODEL_DIR']= 'gs://'+os.environ['BUCKET_NAME']+'/pretrained_model/bert-base-multilingual-uncased'

In [83]:
%%bash
# Use Cloud Machine Learning Engine to train the model on GCP
gcloud ai-platform jobs submit training $JOB_NAME \
        --module-name=$MAIN_TRAINER_MODULE \
        --package-path=$TRAINER_PACKAGE_PATH \
        --staging-bucket=$PACKAGE_STAGING_PATH \
        --config $CONFIG \
        -- \
        --epochs=$EPOCHS \
        --steps_per_epoch_train=$STEPS_PER_EPOCH_TRAIN \
        --batch_size_train=$BATCH_SIZE_TRAIN \
        --steps_per_epoch_eval=$STEPS_PER_EPOCH_EVAL \
        --batch_size_eval=$BATCH_SIZE_EVAL \
        --input_eval_tfrecords=$INPUT_EVAL_TFRECORDS \
        --input_train_tfrecords=$INPUT_TRAIN_TFRECORDS \
        --output_dir=$OUTPUT_DIR \
        --pretrained_model_dir=$PRETRAINED_MODEL_DIR \
        --verbosity_level='DEBUG'

jobId: tf_bert_classification_2020_05_02_151451
state: QUEUED


Job [tf_bert_classification_2020_05_02_151451] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe tf_bert_classification_2020_05_02_151451

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs tf_bert_classification_2020_05_02_151451


## Train the model on AI Platform Training using TPU (for production)

In [26]:
# train on GCP
model_name='tf_bert_classification'
os.environ['JOB_NAME'] = model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['RUNTIME_VERSION'] = '2.1'
os.environ['PYTHON_VERSION'] = '3.7'
os.environ['TRAINER_PACKAGE_PATH'] = os.environ['PYTHONPATH']+'/model'
os.environ['MAIN_TRAINER_MODULE'] = 'model.'+model_name+'.task'
os.environ['REGION_TPU'] = 'europe-west4'

os.environ['USE_TPU'] = 'True' 
os.environ['EPOCHS'] = '1' 
os.environ['STEPS_PER_EPOCH_TRAIN'] = '5' 
os.environ['BATCH_SIZE_TRAIN'] = '32' 
os.environ['STEPS_PER_EPOCH_EVAL'] = '1' 
os.environ['BATCH_SIZE_EVAL'] = '64'
os.environ['PACKAGE_STAGING_PATH'] = 'gs://'+os.environ['BUCKET_STAGING_NAME']
os.environ['INPUT_EVAL_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['INPUT_TRAIN_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['OUTPUT_DIR'] = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['PRETRAINED_MODEL_DIR']= 'gs://'+os.environ['BUCKET_NAME']+'/pretrained_model/bert-base-multilingual-uncased'

In [ ]:
%%bash
# Use Cloud Machine Learning Engine to train the model on GCP
gcloud ai-platform jobs submit training $JOB_NAME \
        --scale-tier basic_tpu\
        --python-version $PYTHON_VERSION \
        --runtime-version $RUNTIME_VERSION \
        --module-name=$MAIN_TRAINER_MODULE \
        --package-path=$TRAINER_PACKAGE_PATH \
        --staging-bucket=$PACKAGE_STAGING_PATH \
        --region=$REGION_TPU \
        -- \
        --use_tpu=$USE_TPU \
        --epochs=$EPOCHS \
        --steps_per_epoch_train=$STEPS_PER_EPOCH_TRAIN \
        --batch_size_train=$BATCH_SIZE_TRAIN \
        --steps_per_epoch_eval=$STEPS_PER_EPOCH_EVAL \
        --batch_size_eval=$BATCH_SIZE_EVAL \
        --input_eval_tfrecords=$INPUT_EVAL_TFRECORDS \
        --input_train_tfrecords=$INPUT_TRAIN_TFRECORDS \
        --output_dir=$OUTPUT_DIR \
        --pretrained_model_dir=$PRETRAINED_MODEL_DIR \
        --verbosity_level='DEBUG' \
#        --stream-logs

## Hyperparameter tunning of the model on AI Platform Training using a config file (for production)

In [61]:
model_name='tf_bert_classification'
os.environ['JOB_NAME'] = model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['TRAINER_PACKAGE_PATH'] = os.environ['PYTHONPATH']+'/model'
os.environ['MAIN_TRAINER_MODULE'] = 'model.'+model_name+'.task'
os.environ['CONFIG']= os.environ['DIR_PROJ'].split('src')[0]+'deployment/hp-tuning/tf_bert_classification/hyperparam.yaml'
os.environ['EPOCHS'] = '1' 
os.environ['STEPS_PER_EPOCH_TRAIN'] = '1' 
os.environ['BATCH_SIZE_TRAIN'] = '32' 
os.environ['STEPS_PER_EPOCH_EVAL'] = '1' 
os.environ['BATCH_SIZE_EVAL'] = '64'
os.environ['PACKAGE_STAGING_PATH'] = 'gs://'+os.environ['BUCKET_STAGING_NAME']
os.environ['INPUT_EVAL_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['INPUT_TRAIN_TFRECORDS'] = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2'
os.environ['OUTPUT_DIR'] = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")
os.environ['PRETRAINED_MODEL_DIR']= 'gs://'+os.environ['BUCKET_NAME']+'/pretrained_model/bert-base-multilingual-uncased'

In [ ]:
ah%%bash
# Use Cloud Machine Learning Engine to train the model on GCP
gcloud ai-platform jobs submit training $JOB_NAME \
        --module-name=$MAIN_TRAINER_MODULE \
        --package-path=$TRAINER_PACKAGE_PATH \
        --staging-bucket=$PACKAGE_STAGING_PATH \
        --config $CONFIG \
        -- \
        --epochs=$EPOCHS \
        --steps_per_epoch_train=$STEPS_PER_EPOCH_TRAIN \
        --batch_size_train=$BATCH_SIZE_TRAIN \
        --steps_per_epoch_eval=$STEPS_PER_EPOCH_EVAL \
        --batch_size_eval=$BATCH_SIZE_EVAL \
        --input_eval_tfrecords=$INPUT_EVAL_TFRECORDS \
        --input_train_tfrecords=$INPUT_TRAIN_TFRECORDS \
        --output_dir=$OUTPUT_DIR \
        --pretrained_model_dir=$PRETRAINED_MODEL_DIR \
        --verbosity_level='INFO'

# TensorBoard for job running on GCP

%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard  --logdir {os.environ['OUTPUT_DIR']+'/tensorboard'} \
              #--port 6001 \
              #--debugger_port 6001

## Description of the job

In [ ]:
%%bash
# remove # to see execute the function
#gcloud ai-platform jobs describe $JOB_NAME

## See the logs

In [ ]:
%%bash
# remove # to see execute the function
#gcloud ai-platform jobs stream-logs $JOB_NAME 

## Options

In [13]:
%%bash
gcloud ai-platform jobs submit training --help

NAME
    gcloud ai-platform jobs submit training - submit an AI Platform training
        job

SYNOPSIS
    gcloud ai-platform jobs submit training JOB [--config=CONFIG]
        [--job-dir=JOB_DIR] [--labels=[KEY=VALUE,...]]
        [--master-accelerator=[count=COUNT],[type=TYPE]]
        [--master-image-uri=MASTER_IMAGE_URI]
        [--master-machine-type=MASTER_MACHINE_TYPE] [--module-name=MODULE_NAME]
        [--package-path=PACKAGE_PATH] [--packages=[PACKAGE,...]]
        [--parameter-server-accelerator=[count=COUNT],[type=TYPE]]
        [--parameter-server-image-uri=PARAMETER_SERVER_IMAGE_URI]
        [--python-version=PYTHON_VERSION] [--region=REGION]
        [--runtime-version=RUNTIME_VERSION] [--scale-tier=SCALE_TIER]
        [--staging-bucket=STAGING_BUCKET]
        [--use-chief-in-tf-config=USE_CHIEF_IN_TF_CONFIG]
        [--worker-accelerator=[count=COUNT],[type=TYPE]]
        [--worker-image-uri=WORKER_IMAGE_URI] [--async | --stream-logs]
        [--parameter-server-count=P